# PyGSTi CHP Object Test

In [1]:
from __future__ import print_function #python 2 & 3 compatibility
import pygsti

from pygsti.objects.operation import LinearOperator, StaticStandardOp, StochasticNoiseOp, DepolarizeOp, ComposedOp, EmbeddedOp
from pygsti.objects.labeldicts import StateSpaceLabels
pygsti.__version__

'0.9.9.2.post715+gcde6e292.d20210315'

## LinearOperator and StaticStandardOp

Now with 'chp' evotype.

In [2]:
Gx = StaticStandardOp('Gxpi', 'chp')
print(Gx)
print(Gx.get_chp_str())
print(Gx.get_chp_str([2]))

StaticStandardOp with name Gxpi and evotype chp
CHP operations: h 0,p 0,p 0,h 0

h 0
p 0
p 0
h 0

h 2
p 2
p 2
h 2



In [3]:
# Can also make custom CHP operations
# Here I'm making a (deterministic) Hadamard on qubit 0 and CNOT on qubits 1 and 2
rep = pygsti.objects.replib.CHPOpRep(['h 0', 'c 1 2'], nqubits=3)
c = LinearOperator(rep, 'chp')

In [4]:
print(c)
print(c.get_chp_str())
print(c.get_chp_str([3,4,5]))

h 0
c 1 2

h 3
c 4 5



In [5]:
print(StaticStandardOp('Gc20', 'chp'))

StaticStandardOp with name Gc20 and evotype chp
CHP operations: p 0,h 0,p 0,p 0,h 0



## StochasticNoiseOp and DepolarizeOp

Now with 'chp' evotype

In [6]:
nqubits = 1
scop = StochasticNoiseOp(4**nqubits, evotype='chp', initial_rates=[0.5, 0.1, 0.1], seed_or_state=2021)
print(scop)
print(scop.get_chp_str([1])) # With seed 2021, pulls Z
print(scop.get_chp_str([2])) # With seed 2021, pulls I (no output)
print(scop.get_chp_str([3])) # With seed 2021, pulls X
print(scop.get_chp_str([4])) # With seed 2021, pulls X

Stochastic noise operation map with dim = 2, num params = 3

p 1
p 1


h 3
p 3
p 3
h 3

h 4
p 4
p 4
h 4



In [7]:
nqubits = 1
dop = DepolarizeOp(4**nqubits, evotype='chp', initial_rate=0.7, seed_or_state=2021)
print(dop)
print(dop.get_chp_str([1])) # With seed 2021, pulls Z
print(dop.get_chp_str([2])) # With seed 2021, pulls I (no output)
print(dop.get_chp_str([3])) # With seed 2021, pulls X
print(dop.get_chp_str([4])) # With seed 2021, pulls Y

Depolarize noise operation map with dim = 2, num params = 1

p 1
p 1


h 3
p 3
p 3
h 3

p 4
h 4
p 4
p 4
h 4
p 4
p 4
p 4



## ComposedOp + EmbeddedOp

In [8]:
# ComposedOp
Gzx_composed = ComposedOp([StaticStandardOp('Gzpi', 'chp'), StaticStandardOp('Gxpi', 'chp')])
print(Gzx_composed)
print(Gzx_composed.get_chp_str())
print(Gzx_composed.get_chp_str([2]))

Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gzpi and evotype chp
CHP operations: p 0,p 0
Factor 1:
StaticStandardOp with name Gxpi and evotype chp
CHP operations: h 0,p 0,p 0,h 0

p 0
p 0
h 0
p 0
p 0
h 0

p 2
p 2
h 2
p 2
p 2
h 2



In [9]:
# EmbeddedOp
Gxi_embedded = EmbeddedOp(['Q0', 'Q1'], ['Q0'], StaticStandardOp('Gxpi', 'chp'))
print(Gxi_embedded)
print(Gxi_embedded.get_chp_str())
print(Gxi_embedded.get_chp_str([5,7]))

Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q0'] space
StaticStandardOp with name Gxpi and evotype chp
CHP operations: h 0,p 0,p 0,h 0

h 0
p 0
p 0
h 0

h 5
p 5
p 5
h 5



In [10]:
Gix_embedded = EmbeddedOp(['Q0', 'Q1'], ['Q1'], StaticStandardOp('Gxpi', 'chp'))
print(Gix_embedded)
print(Gix_embedded.get_chp_str())
print(Gix_embedded.get_chp_str([5,7]))

Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q1'] space
StaticStandardOp with name Gxpi and evotype chp
CHP operations: h 0,p 0,p 0,h 0

h 1
p 1
p 1
h 1

h 7
p 7
p 7
h 7



In [11]:
# EmbeddedOp made of ComposedOps
Gzx_comp_embed = EmbeddedOp(['Q0', 'Q1', 'Q2', 'Q3'], ['Q1'], Gzx_composed)
print(Gzx_comp_embed)
print(Gzx_comp_embed.get_chp_str())
print(Gzx_comp_embed.get_chp_str([5, 6, 7, 8]))

Embedded operation with full dimension 16 and state space Q0(2)*Q1(2)*Q2(2)*Q3(2)
 that embeds the following 2-dimensional operation into acting on the ['Q1'] space
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gzpi and evotype chp
CHP operations: p 0,p 0
Factor 1:
StaticStandardOp with name Gxpi and evotype chp
CHP operations: h 0,p 0,p 0,h 0

p 1
p 1
h 1
p 1
p 1
h 1

p 6
p 6
h 6
p 6
p 6
h 6



## CHPForwardSimulator + Explicit Model

In [12]:
chpexe = '/Users/sserita/Documents/notebooks/pyGSTi/2021-CHP/chp'
sim = pygsti.obj.CHPForwardSimulator(chpexe, shots=100)

In [13]:
#Initialize an empty Model object
model = pygsti.objects.ExplicitOpModel(['Q0', 'Q1'], simulator=sim, evotype='chp')

def make_2Q_op(name0, name1):
    return ComposedOp([
        EmbeddedOp(['Q0', 'Q1'], ['Q0'], StaticStandardOp(name0, 'chp')),
        EmbeddedOp(['Q0', 'Q1'], ['Q1'], StaticStandardOp(name1, 'chp')),
    ])

#Populate the Model object with states, effects, gates,
model['rho0'] = make_2Q_op('Gi', 'Gi')
model['Mdefault'] = pygsti.obj.ComputationalBasisPOVM(2, 'chp')

model['Gii'] = make_2Q_op('Gi', 'Gi')
model['Gxi'] = make_2Q_op('Gxpi', 'Gi')
model['Gix'] = make_2Q_op('Gi', 'Gxpi')
model['Gxx'] = make_2Q_op('Gxpi', 'Gxpi')
model['Gyi'] = make_2Q_op('Gypi', 'Gi')
model['Giy'] = make_2Q_op('Gi', 'Gypi')
model['Gyy'] = make_2Q_op('Gypi', 'Gypi')

print(model)

rho0 = Composed operation of 2 factors:
Factor 0:
Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q0'] space
StaticStandardOp with name Gi and evotype chp
CHP operations: 
Factor 1:
Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q1'] space
StaticStandardOp with name Gi and evotype chp
CHP operations: 


Mdefault = Computational(Z)-basis POVM on 2 qubits and filter None


Gii = 
Composed operation of 2 factors:
Factor 0:
Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q0'] space
StaticStandardOp with name Gi and evotype chp
CHP operations: 
Factor 1:
Embedded operation with full dimension 4 and state space Q0(2)*Q1(2)
 that embeds the following 2-dimensional operation into acting on the ['Q1'] space
StaticStanda

In [14]:
circ = pygsti.obj.Circuit(['Gix'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 1.0),
                  (('10',), 0.0),
                  (('11',), 0.0)])

In [15]:
circ = pygsti.obj.Circuit(['Gix', 'Gxi'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 0.0),
                  (('10',), 0.0),
                  (('11',), 1.0)])

In [16]:
circ = pygsti.obj.Circuit(['rho0', 'Gxx', 'Mdefault'])
model.probabilities(circ)

OutcomeLabelDict([(('00',), 0.0),
                  (('01',), 0.0),
                  (('10',), 0.0),
                  (('11',), 1.0)])

## CHPForwardSimulator + LocalNoiseModel

In [17]:
# Step 1: Define stochastic Pauli noise operators
# Note that the probabilities here are the "error rates" that would be model parameters (currently just static)
noise_1q = StochasticNoiseOp(4, evotype='chp', initial_rates=[0.1, 0.01, 0.01], seed_or_state=2021)

# Also need two-qubit version
# Here we just make it independent stochastic Pauli noise
noise_2q = ComposedOp([EmbeddedOp([0, 1], [0], noise_1q), EmbeddedOp([0, 1], [1], noise_1q)])

In [18]:
# Step 2: Define gate dict of noisy gates
# Using equivalent of XYICNOT modelpack
gatedict = {}
gatedict['Gi'] = noise_1q
gatedict['Gx'] = ComposedOp([StaticStandardOp('Gxpi', 'chp'), noise_1q])
gatedict['Gy'] = ComposedOp([StaticStandardOp('Gypi', 'chp'), noise_1q])
# Note that first Gcnot is now key in model, whereas second Gcnot is a standard gatename known to CHPOp constructor
gatedict['Gcnot'] = ComposedOp([StaticStandardOp('Gcnot', 'chp'), noise_2q])

In [19]:
from pygsti.objects.localnoisemodel import LocalNoiseModel
from pygsti.objects.spamvec import ComputationalSPAMVec

# TODO: Much less convenient to generate parallel gates
rho0 = ComposedOp([EmbeddedOp(range(3), [i], StaticStandardOp('Gi', 'chp')) for i in range(3)])
Mdefault = pygsti.obj.ComputationalBasisPOVM(4, 'chp')

chpexe = '/Users/sserita/Documents/notebooks/pyGSTi/2021-CHP/chp'
sim = pygsti.obj.CHPForwardSimulator(chpexe, shots=100)

ln_model = LocalNoiseModel(num_qubits=4, gatedict=gatedict, prep_layers=rho0, povm_layers=Mdefault,
                           availability={'Gcnot': [(0,1),(1,2),(2,3)]}, simulator=sim, evotype='chp')

In [20]:
# Step 4: Profit?? Worked way too quickly...
def print_implicit_model_blocks(mdl, showSPAM=False):
    if showSPAM:
        print('State prep building blocks (.prep_blks):')
        for blk_lbl,blk in mdl.prep_blks.items():
            print(" " + blk_lbl, ": ", ', '.join(map(str,blk.keys())))
        print()

        print('POVM building blocks (.povm_blks):')
        for blk_lbl,blk in mdl.povm_blks.items():
            print(" "  + blk_lbl, ": ", ', '.join(map(str,blk.keys())))
        print()
    
    print('Operation building blocks (.operation_blks):')
    for blk_lbl,blk in mdl.operation_blks.items():
        print(" " + blk_lbl, ": ", ', '.join(map(str,blk.keys())))
    print()

print_implicit_model_blocks(ln_model, showSPAM=True)

State prep building blocks (.prep_blks):
 layers :  rho0

POVM building blocks (.povm_blks):
 layers :  Mdefault

Operation building blocks (.operation_blks):
 layers :  Gi:0, Gi:1, Gi:2, Gi:3, Gx:0, Gx:1, Gx:2, Gx:3, Gy:0, Gy:1, Gy:2, Gy:3, Gcnot:0:1, Gcnot:1:2, Gcnot:2:3
 gates :  Gi, Gx, Gy, Gcnot



In [21]:
print(ln_model.operation_blks['gates']['Gx'])

Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gxpi and evotype chp
CHP operations: h 0,p 0,p 0,h 0
Factor 1:
Stochastic noise operation map with dim = 2, num params = 3



In [22]:
print(ln_model.operation_blks['layers']['Gcnot', 1, 2])

Embedded operation with full dimension 16 and state space 0(2)*1(2)*2(2)*3(2)
 that embeds the following 4-dimensional operation into acting on the (1, 2) space
Composed operation of 2 factors:
Factor 0:
StaticStandardOp with name Gcnot and evotype chp
CHP operations: c 0 1
Factor 1:
Composed operation of 2 factors:
Factor 0:
Embedded operation with full dimension 4 and state space 0(2)*1(2)
 that embeds the following 2-dimensional operation into acting on the [0] space
Stochastic noise operation map with dim = 2, num params = 3
Factor 1:
Embedded operation with full dimension 4 and state space 0(2)*1(2)
 that embeds the following 2-dimensional operation into acting on the [1] space
Stochastic noise operation map with dim = 2, num params = 3



In [23]:
# Step 5: Actually run circuits with local noise model
# TODO: Marginalized POVMs don't work yet, must specify num_lines as full space
circ = pygsti.obj.Circuit([('Gx', 1)], num_lines=4)
ln_model.probabilities(circ)

OutcomeLabelDict([(('0000',), 0.07),
                  (('0001',), 0.0),
                  (('0010',), 0.0),
                  (('0011',), 0.0),
                  (('0100',), 0.93),
                  (('0101',), 0.0),
                  (('0110',), 0.0),
                  (('0111',), 0.0),
                  (('1000',), 0.0),
                  (('1001',), 0.0),
                  (('1010',), 0.0),
                  (('1011',), 0.0),
                  (('1100',), 0.0),
                  (('1101',), 0.0),
                  (('1110',), 0.0),
                  (('1111',), 0.0)])

In [24]:
circ = pygsti.obj.Circuit([('Gx', 1), ('Gcnot', 1, 2)], num_lines=4)
ln_model.probabilities(circ)

OutcomeLabelDict([(('0000',), 0.06),
                  (('0001',), 0.0),
                  (('0010',), 0.08),
                  (('0011',), 0.0),
                  (('0100',), 0.1),
                  (('0101',), 0.0),
                  (('0110',), 0.76),
                  (('0111',), 0.0),
                  (('1000',), 0.0),
                  (('1001',), 0.0),
                  (('1010',), 0.0),
                  (('1011',), 0.0),
                  (('1100',), 0.0),
                  (('1101',), 0.0),
                  (('1110',), 0.0),
                  (('1111',), 0.0)])

In [25]:
# Could also define correlated noise for 2-qubit error?
pp = pygsti.objects.Basis.cast('pp', 16)
initial_rates = [0.01,]*15
initial_rates[pp.labels.index('XX')] = 0.1 # Set XX to much higher

noise_2q_correlated = StochasticNoiseOp(16, evotype='chp', initial_rates=initial_rates, seed_or_state=2021)

gatedict = {}
gatedict['Gi'] = noise_1q
gatedict['Gx'] = ComposedOp([StaticStandardOp('Gxpi', 'chp'), noise_1q])
gatedict['Gy'] = ComposedOp([StaticStandardOp('Gypi', 'chp'), noise_1q])
# Note that first Gcnot is now key in model, whereas second Gcnot is a standard gatename known to CHPOp constructor
gatedict['Gcnot'] = ComposedOp([StaticStandardOp('Gcnot', 'chp'), noise_2q_correlated])

In [26]:
rho0 = ComposedOp([EmbeddedOp(range(3), [i], StaticStandardOp('Gi', 'chp')) for i in range(3)])
Mdefault = pygsti.obj.ComputationalBasisPOVM(4, 'chp')

chpexe = '/Users/sserita/Documents/notebooks/pyGSTi/2021-CHP/chp'
sim = pygsti.obj.CHPForwardSimulator(chpexe, shots=100)

ln_model_corr = LocalNoiseModel(num_qubits=4, gatedict=gatedict, prep_layers=rho0, povm_layers=Mdefault,
                           availability={'Gcnot': [(0,1),(1,2),(2,3)]}, simulator=sim, evotype='chp')

In [27]:
circ = pygsti.obj.Circuit([('Gx', 1), ('Gcnot', 1, 2)], num_lines=4)
ln_model_corr.probabilities(circ)

OutcomeLabelDict([(('0000',), 0.16),
                  (('0001',), 0.0),
                  (('0010',), 0.02),
                  (('0011',), 0.0),
                  (('0100',), 0.0),
                  (('0101',), 0.0),
                  (('0110',), 0.82),
                  (('0111',), 0.0),
                  (('1000',), 0.0),
                  (('1001',), 0.0),
                  (('1010',), 0.0),
                  (('1011',), 0.0),
                  (('1100',), 0.0),
                  (('1101',), 0.0),
                  (('1110',), 0.0),
                  (('1111',), 0.0)])

In [30]:
circ = pygsti.obj.Circuit([('Gx', 1), ('Gcnot', 1, 2)], num_lines=4)
ln_model_corr.probabilities(circ)

OutcomeLabelDict([(('0000',), 0.21),
                  (('0001',), 0.0),
                  (('0010',), 0.04),
                  (('0011',), 0.0),
                  (('0100',), 0.03),
                  (('0101',), 0.0),
                  (('0110',), 0.72),
                  (('0111',), 0.0),
                  (('1000',), 0.0),
                  (('1001',), 0.0),
                  (('1010',), 0.0),
                  (('1011',), 0.0),
                  (('1100',), 0.0),
                  (('1101',), 0.0),
                  (('1110',), 0.0),
                  (('1111',), 0.0)])